In [14]:
import sys
sys.path.append('../../')
import symlearn.algorithms.r2n.simple_rulenet as rn
import symlearn.algorithms.r2n.base as base
import symlearn.algorithms.r2n.utilities as util
import symlearn.algorithms.r2n.preprocessing_layers as ul
import symlearn.algorithms.r2n.r2n_algo as algo
from symlearn.algorithms.r2n.training import train as train_R2N
import symlearn.trxf.core.utils as trxfutils

In [15]:
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler

import numpy as np
import matplotlib.pylab as plt

torch.manual_seed(2)
np.random.seed(2)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# Control variables
data_loc = 'data/toy_example.csv'
label_col = 'y'
to_drop = []
pos_class = True     # Used only if label_transform = to_binary

# Prepare the data
data = pd.read_csv(data_loc)
data = data.drop(to_drop, axis=1)
unique_labels = data[label_col].unique().tolist()
data[label_col] = data[label_col]
input_data = data.drop(columns=['y', 'Unnamed: 0'])
input_data[['Z_0','Z_1']]= input_data[['Z_0','Z_1']].replace(0,'No')
input_data[['Z_0','Z_1']]= input_data[['Z_0','Z_1']].replace(1,'Yes')
label = data[['y']]

In [17]:
#input_data = util.preprocessing_input(input_data)
input_train,input_test,label_train, label_test = train_test_split(input_data, label,train_size=0.75)

In [18]:
R2run = algo.R2Nalgo(n_seeds=1, decay_rate=0.9, coef = 10**-3, normalize_num=False)

In [19]:
R2run.fit(input_train, label_train)

5 numerical features and  2 categorical features
---------
4 categorical dummies
---------
Epoch: 0 Success rate: 9.3 Loss: 69.27712 Temperature: 9.0e-01
Epoch: 10 Success rate: 90.7 Loss: 7.05005 Temperature: 3.1e-01
Epoch: 20 Success rate: 97.5 Loss: 1.83077 Temperature: 1.1e-01
Epoch: 30 Success rate: 99.6 Loss: 0.68996 Temperature: 3.8e-02
Epoch: 40 Success rate: 99.8 Loss: 0.44184 Temperature: 1.3e-02
Epoch: 50 Success rate: 99.7 Loss: 0.46140 Temperature: 4.6e-03
Epoch: 60 Success rate: 99.7 Loss: 0.44051 Temperature: 1.6e-03
Epoch: 70 Success rate: 99.8 Loss: 0.43708 Temperature: 5.6e-04
Epoch: 80 Success rate: 99.8 Loss: 0.40336 Temperature: 2.0e-04
Epoch: 90 Success rate: 99.7 Loss: 0.52863 Temperature: 9.4e-05
Epoch: 100 Success rate: 99.8 Loss: 0.43973 Temperature: 9.4e-05
---------
Max performance of the network: 99.97333333333333


In [20]:
prediction = R2run.predict(input_test)

In [21]:
R2run.opt_rule

[array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0])]

In [22]:
np.count_nonzero(np.array(prediction).flatten() == np.array(label_test).flatten())/len(label_test)

0.998

In [23]:
print(str(R2run.export_rules_to_trxf_dnf_ruleset(thr=0.1)))

if
([0.42063054*X_1 + 0.4244901*X_3 - 0.20981455 >= 0]) v
([0.43276927*X_0 - 0.21856208 >= 0] ^ [Z_1 == No])
then
Churn


In [24]:
print(str(R2run.export_rules_to_trxf_dnf_ruleset(thr=0.1)))

if
([0.42063054*X_1 + 0.4244901*X_3 - 0.20981455 >= 0]) v
([0.43276927*X_0 - 0.21856208 >= 0] ^ [Z_1 == No])
then
Churn


In [25]:
print(str(R2run.export_rules_to_trxf_dnf_ruleset(thr=0.1)))

if
([0.42063054*X_1 + 0.4244901*X_3 - 0.20981455 >= 0]) v
([0.43276927*X_0 - 0.21856208 >= 0] ^ [Z_1 == No])
then
Churn


In [26]:
trxfutils.batch_evaluate(R2run.export_rules_to_trxf_dnf_ruleset(thr=0.1), input_data)

0        True
1        True
2        True
3       False
4        True
        ...  
9995     True
9996    False
9997     True
9998     True
9999     True
Length: 10000, dtype: bool

In [27]:
input_data

,X_0,X_1,X_2,X_3,X_4,Z_0,Z_1
0,0.967030,0.547232,0.972684,0.714816,0.697729,No,No
1,0.216089,0.976274,0.006230,0.252982,0.434792,Yes,No
2,0.779383,0.197685,0.862993,0.983401,0.163842,No,No
3,0.597334,0.008986,0.386571,0.044160,0.956653,No,Yes
4,0.436147,0.948977,0.786306,0.866289,0.173165,No,Yes
...,...,...,...,...,...,...,...
9995,0.968924,0.467682,0.090192,0.448410,0.538218,Yes,No
9996,0.185715,0.355653,0.496808,0.048329,0.853294,Yes,No
9997,0.371479,0.388439,0.480998,0.370167,0.314440,No,Yes
9998,0.639484,0.175788,0.390954,0.759243,0.397949,Yes,Yes
